# IMPORTS

In [1]:
import sys
sys.path.insert(0, "../..")
import config as cfg
import gc

In [2]:
import pandas as pd
import numpy as np
import re
from tqdm.notebook import tqdm
from metrics import *
from helper import *
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [3]:
import catboost as cb
import catboost.datasets as cbd
import catboost.utils as cbu

# MODEL TRAINING

In [4]:
train = pd.read_pickle(cfg.PREPARED_TRAIN_DATA_PATH)
test = pd.read_pickle(cfg.PREPARED_TEST_DATA_PATH)

In [5]:
X_train, Y_train = train.drop(cfg.TARGETS, axis=1), train[cfg.TARGETS]
ORIGINAL_COLS = X_train.columns

In [6]:
model_names = {
    'logreg': ['baseline'],
    # 'mlp': ['baseline'],
    # 'svm': ['baseline'],
    # 'gbt': ['baseline'],
    # 'rf': ['baseline'],
    'catboost': ['baseline', 'only_real_cols'] # , 'only_cat_cols'
}

In [7]:
X_train = X_train.join(asemble_data(cfg.OOF_PRED_PATH, model_names))
test = test.join(asemble_data(cfg.TEST_PRED_PATH, model_names))

In [8]:
assert X_train.columns.tolist() == test.columns.tolist()

In [9]:
pred_proba_oof = pd.DataFrame(data=np.zeros(shape=(len(train), len(cfg.TARGETS))), index=train.index, columns=cfg.TARGETS)
pred_proba_test = pd.DataFrame(data=np.zeros(shape=(len(test), len(cfg.TARGETS))), index=test.index, columns=cfg.TARGETS)
metrics = {}

In [10]:
EXPERIMENT_FAMILY_NAME = 'stacking'
EXPERIMENT_NAME = 'baseline'
RANDOM_STATE = 77
N_SPLITS = 5
N_RANDOM_SEEDS = 7

In [11]:
X_train, test = X_train.drop(ORIGINAL_COLS, axis=1), test.drop(ORIGINAL_COLS, axis=1)

In [12]:
cv = MultilabelStratifiedKFold(n_splits=N_SPLITS, random_state=RANDOM_STATE, shuffle=True)

CAT_COLS = cfg.CAT_UNORDERED_COLS

test_pool = cb.Pool(
        data=test,
        # cat_features=CAT_COLS
        )

fold = 0
for train_idx, val_idx in tqdm(cv.split(X_train, Y_train), total=N_SPLITS):

    train_pool = cb.Pool(
        data=X_train.iloc[train_idx], 
        label=Y_train.iloc[train_idx],
        # cat_features=CAT_COLS
        )

    val_pool = cb.Pool(
        data=X_train.iloc[val_idx], 
        label=Y_train.iloc[val_idx],
        # cat_features=CAT_COLS
        )
        

    clf = cb.CatBoostClassifier(
        loss_function='MultiLogloss',
        custom_metric=['Recall', 'F1'],
        silent=True,
        iterations=1200,
        depth=3,
        # l2_leaf_reg=5.0,
        learning_rate=0.0001,
        early_stopping_rounds=100,
        random_strength=2.5,
        bagging_temperature=1
    )

    clf.fit(train_pool, eval_set=val_pool, plot=False)
    
    model_name = f'{EXPERIMENT_NAME}_fold_{fold}.pkl'
    model_path = os.path.join(cfg.MODELS_PATH, EXPERIMENT_FAMILY_NAME, EXPERIMENT_NAME)
    check_path(model_path)
    clf.save_model(os.path.join(model_path, model_name))
    
    pred_proba_oof.iloc[val_idx, :] += clf.predict_proba(val_pool)
    pred_proba_test.iloc[:, :] += clf.predict_proba(test_pool)
    gc.collect()

    fold += 1
    
pred_proba_test /= N_SPLITS

  0%|          | 0/5 [00:00<?, ?it/s]

# PREDICT AND SAVE PREDICTIONS

In [13]:
tresholds = get_tresholds(train[cfg.TARGETS], pred_proba_oof)
sample_submission = pd.read_csv(cfg.SAMPLE_SUBMISSION_PATH).set_index('ID')
submission = make_prediction(pred_proba_test, tresholds, sample_submission)

0.6858488948888258


In [14]:
## BEST PARAMS
# 0.6912250203448425

# model_names = ['catboost', 'logreg', 'rf', 'mlp'] # 'gbt',  
# experiments = ['baseline']

# cv = MultilabelStratifiedKFold(n_splits=N_SPLITS, random_state=RANDOM_STATE, shuffle=True)

# CAT_COLS = cfg.CAT_UNORDERED_COLS

# test_pool = cb.Pool(
#         data=test,
#         cat_features=CAT_COLS
#         )

# fold = 0
# for train_idx, val_idx in tqdm(cv.split(X_train, Y_train), total=N_SPLITS):

#     train_pool = cb.Pool(
#         data=X_train.iloc[train_idx], 
#         label=Y_train.iloc[train_idx],
#         cat_features=CAT_COLS
#         )

#     val_pool = cb.Pool(
#         data=X_train.iloc[val_idx], 
#         label=Y_train.iloc[val_idx],
#         cat_features=CAT_COLS
#         )
        

#     clf = cb.CatBoostClassifier(
#         loss_function='MultiLogloss',
#         custom_metric=['Recall', 'F1'],
#         silent=True,
#         iterations=1200,
#         depth=2,
#         l2_leaf_reg=5.0,
#         learning_rate=0.0001,
#         early_stopping_rounds=100
#     )

#     clf.fit(train_pool, eval_set=val_pool, plot=False)
    
#     model_name = f'{EXPERIMENT_NAME}_fold_{fold}.pkl'
#     model_path = os.path.join(cfg.MODELS_PATH, EXPERIMENT_FAMILY_NAME, EXPERIMENT_NAME)
#     check_path(model_path)
#     clf.save_model(os.path.join(model_path, model_name))
    
#     pred_proba_oof.iloc[val_idx, :] += clf.predict_proba(val_pool)
#     pred_proba_test.iloc[:, :] += clf.predict_proba(test_pool)
#     gc.collect()

#     fold += 1
    
# pred_proba_test /= N_SPLITS

In [15]:
submission_path = os.path.join(cfg.SUBMISSION_PATH, EXPERIMENT_FAMILY_NAME)
check_path(submission_path)
submission.to_csv(os.path.join(submission_path, f'{EXPERIMENT_NAME}.csv'))

pred_proba_oof_path = os.path.join(cfg.OOF_PRED_PATH, EXPERIMENT_FAMILY_NAME)
check_path(pred_proba_oof_path)
pred_proba_oof.to_pickle(os.path.join(pred_proba_oof_path, f'{EXPERIMENT_NAME}.pkl'))

pred_proba_test_path = os.path.join(cfg.TEST_PRED_PATH, EXPERIMENT_FAMILY_NAME)
check_path(pred_proba_test_path)
pred_proba_test.to_pickle(os.path.join(pred_proba_test_path, f'{EXPERIMENT_NAME}.pkl'))